# Ensemblify Quick Reference Guide

Ensemblify is a library used for generating and analyzing protein conformational ensembles of proteins.

This notebook will walk you through a typical Ensemblify workflow, with basic usage examples for all available modules.

# `generation` module

## 1. Setting up your parameters file

Before generating a conformational ensemble for your protein of interest, you must first create an Ensemblify parameters file, optionally using the provided HTML form.

You must provide at least:

- **job_name**: prefix identifier for files and directories created during ensemble generation.

- **sequence**: the sequence or structure of your protein of interest. This can be a sequence string, the path to a text file containing that sequence string, the path to a PDB file of your protein or a UniProt Acession Number if you want Ensemblify to automatically fetch the AlphaFold prediction for your protein from the AlphaFold Protein Structure Database.

- **size**: the number of alternate conformations of your protein you want to include in the ensemble.

- **databases**: a mapping of database IDs to the path where each database is stored. You must provide at least one database to sample dihedral angles from.

- **targets**: a mapping of protein chain letters found in your input PDB file to a list of target sampling regions. Each sampling region is defined by its range of residue numbers (e.g. [1,50]), database to sample from (i.e. a database ID present in **databases**) and sampling mode ('TRIPEPTIDE' or 'SINGLERESIDUE').

- **output_path**: the path to the directory where Ensemblify will store the created ensemble. This parameter defaults to the current working directory, so we recommend that you choose an appropriate location.

You can check the provided parameters file template for an example. We can read and display its contents:

In [2]:
import yaml
import pprint

HST5_PARAMS = 'input_parameters/generate_Hst5.yaml'

# read
with open(HST5_PARAMS) as parameters_file:
    parameters = yaml.safe_load(parameters_file)

# display
pprint.pprint(parameters,sort_dicts=False,compact=True)

{'job_name': 'Hst5_test',
 'sequence': 'DSHAKRHHGYKRKFHEKHHSHRGY',
 'size': 10,
 'databases': {'coil': '/home/tiagogomes/Desktop/projects/nuno_fernandes/tripeptide_db_moma_cortes/tripeptide_angles_data_UPDATED/tripeptide_angles_data_BEST_FORMATS/coil.parquet'},
 'targets': {'A': [['MC', [1, 24], 'coil', 'TRIPEPTIDE']]},
 'output_path': '.'}


## 2. Generating a conformational ensemble

To generate a conformational ensemble for your protein of interest, you simply need to provide Ensemblify with your parameters file.

For this example, we will use a parameters file created to sample Histatin5, a small IDP that can normally be found in saliva, where it acts as a defense against fungal infections.

If you want to run this example on your machine, you must update in the parameters file the location of the database you wish to sample from.

In [3]:
from ensemblify.generation import generate_ensemble

generated_ensemble_directory = generate_ensemble(parameters_path=HST5_PARAMS)

Generating ensemble of 10 valid pdbs using 31 processor cores... 


Ensemblified!: 100%|██████████| 10/10 [00:06<00:00,  1.43valid_pdb/s]   

There are 10 valid pdbs, 5 were discarded ( 5 clashed | 0 violated constraints).
Ensemble Generation Finished!


We can then check the location of our freshly generated ensemble.

In [4]:
print(generated_ensemble_directory)

./Hst5_test/ensemble/valid_pdbs


# `conversion` module

## 1. Converting an ensemble to a trajectory

After generating an ensemble, you can convert your set of PDB files into XTC format, a compressed **trajectory** format from GROMACS. This allows for much more efficient storage of created ensembles with minimal loss of structural information. Additionally, we can take advantage of the many available methods for analyzing files in this format.

To do so, you need only provide the location of your stored ensemble and the directory where you want to store your trajectory, optionally defining a prefix identifier for the created file.

Along with the created trajectory, one of the structures of the generated ensemble is saved as a **topology** PDB file that contains atomic connectivity information, as this is not stored in the trajectory and is required when using the `analysis` module.

In [5]:
from ensemblify.conversion import ensemble2traj

TRAJECTORY_DESTINATION = 'Hst5_test/trajectory'
TRAJECTORY_ID = 'Hst5'

trajectory_file, topology_file = ensemble2traj(ensemble_dir=generated_ensemble_directory,
                                               trajectory_dir=TRAJECTORY_DESTINATION,
                                               trajectory_id=TRAJECTORY_ID)

 Hst5 Trajectory creation complete! : 100%|██████████| 4/4 [00:00<00:00, 566.26step/s]


We can then check the locations of your created trajectory and topology files.

In [6]:
print(trajectory_file)
print(topology_file)

Hst5_test/trajectory/Hst5_trajectory.xtc
Hst5_test/trajectory/Hst5_top.pdb


# `analysis` module

# 1. Analyzing your created trajectory

After creating your trajectory, you can use it to create an interactive analysis dashboard with different plots and figures which will aid you in the structural analysis of your protein using your created ensemble.

To do this, specify the location of your **trajectory** and **topology** files and the output directory where you want to store your interactive dashboard and the figures and data used in its creation.

In [7]:
from ensemblify.analysis import analyze_trajectory

TRAJECTORY_ANALYSIS_OUTPUT = 'Hst5_test/trajectory_analysis'

analysis_data = analyze_trajectory(trajectories=trajectory_file,
                                   topologies=topology_file,
                                   trajectory_ids=TRAJECTORY_ID,
                                   output_directory=TRAJECTORY_ANALYSIS_OUTPUT)

Analyzing Hst5 trajectory...
Calculating ramachandran data for Hst5...
Calculating contact matrix for Hst5...


Calculating contact matrix...: 100%|██████████| 10/10 [00:00<00:00, 1576.87it/s]

Calculating distance matrix for Hst5...



Calculating distance matrix... : 100%|██████████| 10/10 [00:00<00:00, 2161.57it/s]

Calculating secondary structure assignment frequency matrix for Hst5...
Calculating structural metrics data for Hst5...
Calculating rg...


Calculating eed...
Calculating dmax...
Creating Hst5 analysis figures...
Building ['Hst5'] analysis dashboard...
Ensemble analysis calculation has finished. Please consult the interactive analysis_dashboard.html figure.


We can then open our analysis_dashboard HTML file in a web browser and interpret our results.

# `reweighting` module

## 1. Reweighting your ensemble with experimental SAXS data

After generating an ensemble, you can use experimental SAXS data to reweight it.

In [8]:
from ensemblify.reweighting import reweight_ensemble

EXP_SAXS_DATA_FILE = 'SAXS_data/bift_Hst5.dat'
ENSEMBLE_REWEIGHTING_OUTPUT = 'Hst5_test/reweighting'

reweight_ensemble(trajectory=trajectory_file,
                  topology=topology_file,
                  trajectory_id=TRAJECTORY_ID,
                  exp_saxs_data=EXP_SAXS_DATA_FILE,
                  output_dir=ENSEMBLE_REWEIGHTING_OUTPUT,)

Processing Hst5 experimental data file...
Experimental errors on SAXS intensities have been corrected with BIFT using scale factor 1.0.


Calculating Hst5 SAXS data... : 100%|██████████| 10/10 [00:00<00:00, 46.02it/s]

Applying BME reweighting to Hst5 ensemble with different values for theta parameter...



Reweighting ensemble... : 100%|██████████| 12/12 [00:00<00:00, 143.69it/s]

Please analyze the provided interactive figure (effective_frames_fit.html) and input the desired value(s) for the theta parameter.
If more than one value, please separate them using a comma.


...

Calculating reweighted contact matrix...: 100%|██████████| 10/10 [00:00<00:00, 951.91it/s]

No distance matrix data was provided.



Calculating reweighted distance matrix... : 100%|██████████| 10/10 [00:00<00:00, 2204.98it/s]

No secondary structure assignment frequency matrix data was provided.
Calculating reweighted secondary structure assignment frequency matrix...
No structural metrics distributions data was provided.
Calculating rg...


Calculating eed...
Calculating dmax...
Creating Hst5 reweighted interactive figures...
Building Hst5 reweighting dashboard...
Ensemble reweighting has finished. Please refer to the interactive reweighting_dashboard.html figure for analysis.
